<a href="https://colab.research.google.com/github/CassDabii/BBC-DS-Task/blob/main/BBC_DSProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BBC Data Science Project**
---
Since this project is open ended it is up to me to determine what data is useful to make actionable insights. To do this I will make a list of *preliminary* goals that are variable, however any goals added or omitted will not be hidden but instead justified to maintain credibility




Goals
*   Determine the effect weather has on the length of the journey and the cycle volume.
*   Determine the usefulness and performance of trying to predict cycle volumes
*   Decipher where to add another station
*   If the business was ever wanting to get rid of a station what station would make the most sense to get rid of.
*   What effect bike station capacity has (e.g., the more spaces the more people go use it).
*   Which stations would it be useful to expand.












## Acquiring Data

I use the inital cell to declare my imports because it makes the notebook more organised and I never have to look for where I imported something.


In [112]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sqlite3 

In [113]:
conn = sqlite3.connect('/content/BBCDS.sqlite3')

cursor = conn.cursor()

In [ ]:
bike_journeys = pd.read_sql('SELECT * FROM bike_journeys;',conn)
bike_stations = pd.read_sql('SELECT * FROM bike_stations;',conn)
weather = pd.read_sql('SELECT * FROM weather;',conn)

### Bike Journeys

***COLUMNS ARE MISSING FROM THE DATABASE BECAUSE LATER(UNDER THE BIKE JOURNEYS AND STATIONS PREPARATIONS) I DROP SOME COLUMNS ***

In [115]:
bike_journeys # Shows the data in a dataframe format 

,Journey Duration,Journey ID,End Hour,End Minute,End Station ID,Start Hour,Start Minute,Start Station ID,formatted_start_date,formatted_end_date
0,2040,953,18,0,478,17,26,251,19/09/2017,19/09/2017
1,1800,12581,15,21,122,14,51,550,19/09/2017,19/09/2017
2,1140,1159,17,1,639,16,42,212,15/09/2017,15/09/2017
3,420,2375,12,16,755,12,9,163,14/09/2017,14/09/2017
4,1200,14659,19,33,605,19,13,36,13/09/2017,13/09/2017
...,...,...,...,...,...,...,...,...,...,...
1542839,270,5296,0,2,32,23,58,42,07/08/2017,08/08/2017
1542840,911,12348,0,13,625,23,58,222,07/08/2017,08/08/2017
1542841,447,8303,0,7,453,23,59,130,07/08/2017,08/08/2017
1542842,424,12038,0,6,405,23,59,755,07/08/2017,08/08/2017


I output the dataframe just by using the variable to see the columns, the size of the data and get some preliminary domain knowledge. Moreover, the reason I use this method is because the nodebook workspace I use has a wand icon next to the dataframe that allows me to sort in acending or descending order through each column without using any code which is a lot more seamless. Cross referencing the value for journey duration and start and ending times I now know that the journey duration is measured in seconds.

In [116]:
bike_journeys.dtypes # Shows the data type for each feature/varible

Journey Duration         int64
Journey ID               int64
End Hour                 int64
End Minute               int64
End Station ID           int64
Start Hour               int64
Start Minute             int64
Start Station ID         int64
formatted_start_date    object
formatted_end_date      object
dtype: object

Since these are all numerical I am thinking the use of the .corr() will be useful for initial inteprettions.

### Bike Stations

In [117]:
bike_stations

,Station ID,Capacity,Latitude,Longitude,Station Name
0,1,19,51.529163,-0.109970,"River Street , Clerkenwell"
1,2,37,51.499606,-0.197574,"Phillimore Gardens, Kensington"
2,3,32,51.521283,-0.084605,"Christopher Street, Liverpool Street"
3,4,23,51.530059,-0.120973,"St. Chad's Street, King's Cross"
4,5,27,51.493130,-0.156876,"Sedding Street, Sloane Square"
...,...,...,...,...,...
768,190,21,51.489975,-0.132845,"Rampayne Street, Pimlico"
769,194,56,51.504627,-0.091773,"Hop Exchange, The Borough"
770,195,30,51.507244,-0.106237,"Milroy Walk, South Bank"
771,196,17,51.503688,-0.098497,"Union Street, The Borough"


I can already see that the Station ID can is a foreign key between bike stations and bike journeys. I am expecting to use a join to get all the data in one flat table.

In [118]:
bike_stations.dtypes

Station ID        int64
Capacity          int64
Latitude        float64
Longitude       float64
Station Name     object
dtype: object

### Weather

In [119]:
weather.head()

,LATITUDE,LONGITUDE,DATE,PRCP (MM),TAVG (CELSIUS)
0,51.478,-0.461,01/08/2017,0.0,17.1
1,51.478,-0.461,02/08/2017,0.8,16.8
2,51.478,-0.461,03/08/2017,7.1,18.4
3,51.478,-0.461,04/08/2017,0.0,18.3
4,51.478,-0.461,05/08/2017,0.0,16.8


The way the date is formated in this dataset differs from the bike journeys data set so this will have to be changed so I can join these datasets. Also this dataset includes dates from august when all of the bike journeys take place in september so that will be omitted as it is not needed. Also, this data only tells of the weather in one area and tells us the precipitation and temperature for the day not detailing wether it was the average or the highest respective values for that day and not giving specific times.

In [120]:
weather.dtypes

LATITUDE          float64
LONGITUDE         float64
DATE               object
PRCP (MM)         float64
TAVG (CELSIUS)    float64
dtype: object

## Prepare

In [124]:
bike_journeys.isnull().any() # The use of these 2 methods together checks if there are missing values in any of the columns

Journey Duration        False
Journey ID              False
End Hour                False
End Minute              False
End Station ID          False
Start Hour              False
Start Minute            False
Start Station ID        False
formatted_start_date    False
formatted_end_date      False
dtype: bool

Initally I want to check if there are any missing data in any of the columns since it could most probably be lowering the data quality. However, other alternatives to just droping the rows of data could be looked into.Depending on factors such as where the missing data lies (e.g, if the station ID's are missing that is major factor for the outcomes but if the start hour is missing but there is still the journey duration the data could be filled.) this also depends on how much missing data there is could tell me if it worth going through these changes.

In [125]:
bike_stations.isnull().any()

Station ID      False
Capacity        False
Latitude        False
Longitude       False
Station Name    False
dtype: bool

In [126]:
weather.isnull().any()

LATITUDE          False
LONGITUDE         False
DATE              False
PRCP (MM)         False
TAVG (CELSIUS)    False
dtype: bool

### Bike Journey and Stations Preparations

In [136]:
journey_station =pd.read_sql('''SELECT *
               FROM bike_journeys
               JOIN bike_stations ON bike_journeys.[Start Station ID] = bike_stations.[Station ID]''',conn)
journey_station

,Journey Duration,Journey ID,End Hour,End Minute,End Station ID,Start Hour,Start Minute,Start Station ID,formatted_start_date,formatted_end_date,Station ID,Capacity,Latitude,Longitude,Station Name
0,2040,953,18,0,478,17,26,251,19/09/2017,19/09/2017,251,34,51.518908,-0.079249,"Brushfield Street, Liverpool Street"
1,1800,12581,15,21,122,14,51,550,19/09/2017,19/09/2017,550,23,51.521564,-0.039264,"Harford Street, Mile End"
2,1140,1159,17,1,639,16,42,212,15/09/2017,15/09/2017,212,17,51.506584,-0.199004,"Campden Hill Road, Notting Hill"
3,420,2375,12,16,755,12,9,163,14/09/2017,14/09/2017,163,27,51.493184,-0.167894,"Sloane Avenue, Knightsbridge"
4,1200,14659,19,33,605,19,13,36,13/09/2017,13/09/2017,36,28,51.501737,-0.184980,"De Vere Gardens, Kensington"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530235,270,5296,0,2,32,23,58,42,07/08/2017,08/08/2017,42,28,51.530991,-0.093903,"Wenlock Road , Hoxton"
1530236,911,12348,0,13,625,23,58,222,07/08/2017,08/08/2017,222,43,51.502757,-0.155349,"Knightsbridge, Hyde Park"
1530237,447,8303,0,7,453,23,59,130,07/08/2017,08/08/2017,130,24,51.509506,-0.075459,"Tower Gardens , Tower"
1530238,424,12038,0,6,405,23,59,755,07/08/2017,08/08/2017,755,24,51.485121,-0.174971,"The Vale, Chelsea"


I join the bike journeys and bike stations tables to have all the data in one flat table so I can use methods like .corr and or visualise correlations and other statistical methods. I Know that I will have to join the weather dataset at somee point but there isn't a give column to perform a join. A merge using pandas is posible but I will have to cross reference so I write a SQL script in the sqlite3 interface that turns the date columns into the date format DD/MM/YYYY.




**Adds the columns to the table**
```
ALTER TABLE bike_journeys ADD COLUMN formatted_start_date TEXT;
ALTER TABLE bike_journeys ADD COLUMN formatted_end_date TEXT;
```



**This uses the columns to make the date in the DD/MM/YYYY format and if the month or date is a single character it adds 0 to the front. 2000 is added to the year to fit the format.**
```
UPDATE bike_journeys
SET formatted_start_date = CASE
  WHEN LENGTH("Start Date") = 1 AND LENGTH("Start Month") = 1 THEN '0'||"Start Date"||'/0'||"Start Month"||'/'||("Start Year"+2000)
  WHEN LENGTH("Start Date") = 1 THEN '0'||"Start Date"||'/'||"Start Month"||'/'||("Start Year"+2000)
  WHEN LENGTH("Start Month") = 1 THEN "Start Date"||'/0'||"Start Month"||'/'||("Start Year"+2000)
  ELSE "Start Date"||'/'||"Start Month"||'/'||("Start Year"+2000)
END,
formatted_end_date = CASE
  WHEN LENGTH("End Date") = 1 AND LENGTH("End Month") = 1 THEN '0'||"End Date"||'/0'||"End Month"||'/'||("End Year"+2000)
  WHEN LENGTH("End Date") = 1 THEN '0'||"End Date"||'/'||"End Month"||'/'||("End Year"+2000)
  WHEN LENGTH("End Month") = 1 THEN "End Date"||'/0'||"End Month"||'/'||("End Year"+2000)
  ELSE "End Date"||'/'||"End Month"||'/'||("End Year"+2000)
END;

```

**Then the old dates are dropped**

```
ALTER TABLE bike_journeys
DROP COLUMN "End Date",
DROP COLUMN "End Month",
DROP COLUMN "End Year",
DROP COLUMN "Start Date",
DROP COLUMN "Start Month",
DROP COLUMN "Start Year";
```









### Weather Preparation

In [128]:
pd.read_sql('''SELECT * 
                FROM bike_stations
                WHERE [Latitude] = "51.478" AND [Longitude] = "-0.461"
                ;''', conn) # Check the row where the weather has the exact same latitude and logitude 

,Station ID,Capacity,Latitude,Longitude,Station Name


There is no row with this exact latitdue and logitude so I have to check how close the nearest station is to this point and if the distance is large enough to disregard the weather data. 

In [129]:
pd.read_sql('''SELECT *
                FROM bike_stations
                ORDER BY ABS([Latitude] - 51.478) + ABS([Longitude] + 0.461)
                ;''', conn)
# This query shows orders the closest station to the furtherst station from the given longitude and latitude 

,Station ID,Capacity,Latitude,Longitude,Station Name
0,668,26,51.494223,-0.236769,"Ravenscourt Park Station, Hammersmith"
1,753,28,51.492636,-0.234094,"Hammersmith Town Hall, Hammersmith"
2,644,36,51.483732,-0.223852,"Rainville Road, Hammersmith"
3,682,46,51.488108,-0.226606,"Crisp Road, Hammersmith"
4,599,28,51.485743,-0.223616,"Manbre Road, Hammersmith"
...,...,...,...,...,...
768,785,64,51.540940,-0.010510,"Aquatic Centre, Queen Elizabeth Olympic Park"
769,787,35,51.546805,-0.014691,"Timber Lodge, Queen Elizabeth Olympic Park"
770,786,44,51.549369,-0.015717,"Lee Valley VeloPark, Queen Elizabeth Olympic Park"
771,784,34,51.546326,-0.009935,"East Village, Queen Elizabeth Olympic Park"


Using an online google maps I calculated the distance to the furthest station and the given longitude and latitude in the weather dataset (20.11 miles). This still falls under the london GPE so the weather could be taken under consideration with every journey. The time not being a part of of the date is still an issue.

In [130]:
weather = pd.read_sql('''SELECT * FROM weather WHERE [DATE] LIKE "%/09/2017";''', conn)
weather

,LATITUDE,LONGITUDE,DATE,PRCP (MM),TAVG (CELSIUS)
0,51.478,-0.461,01/09/2017,1.5,16.1
1,51.478,-0.461,02/09/2017,0.0,15.8
2,51.478,-0.461,03/09/2017,0.0,13.7
3,51.478,-0.461,04/09/2017,6.1,17.7
4,51.478,-0.461,05/09/2017,0.3,17.6
5,51.478,-0.461,06/09/2017,1.5,15.8
6,51.478,-0.461,07/09/2017,0.0,15.8
7,51.478,-0.461,08/09/2017,0.8,15.8
8,51.478,-0.461,09/09/2017,6.1,13.2
9,51.478,-0.461,10/09/2017,7.1,13.3


In [134]:
combined_data = pd.read_sql('''SELECT *
                               FROM bike_journeys
                               JOIN bike_stations ON bike_journeys.[Start Station ID] = bike_stations.[Station ID]
                               JOIN weather ON bike_journeys.formatted_start_date = weather.DATE;
                               ''', conn)
combined_data = combined_data.drop(columns=["DATE", "LONGITUDE", "LATITUDE"])
combined_data

,Journey Duration,Journey ID,End Hour,End Minute,End Station ID,Start Hour,Start Minute,Start Station ID,formatted_start_date,formatted_end_date,Station ID,Capacity,Latitude,Longitude,Station Name,PRCP (MM),TAVG (CELSIUS)
0,1200,14659,19,33,605,19,13,36,13/09/2017,13/09/2017,36,28,51.501737,-0.184980,"De Vere Gardens, Kensington",2.5,14.8
1,900,9923,8,32,298,8,17,154,13/09/2017,13/09/2017,154,35,51.503791,-0.112824,"Waterloo Station 3, Waterloo",2.5,14.8
2,480,9642,15,31,194,15,23,156,13/09/2017,13/09/2017,156,33,51.494436,-0.092921,"New Kent Road, The Borough",2.5,14.8
3,480,12668,15,31,194,15,23,156,13/09/2017,13/09/2017,156,33,51.494436,-0.092921,"New Kent Road, The Borough",2.5,14.8
4,660,8907,8,42,368,8,31,307,13/09/2017,13/09/2017,307,24,51.509908,-0.187842,"Black Lion Gate, Kensington Gardens",2.5,14.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353589,270,5296,0,2,32,23,58,42,07/08/2017,08/08/2017,42,28,51.530991,-0.093903,"Wenlock Road , Hoxton",0.0,16.7
1353590,911,12348,0,13,625,23,58,222,07/08/2017,08/08/2017,222,43,51.502757,-0.155349,"Knightsbridge, Hyde Park",0.0,16.7
1353591,447,8303,0,7,453,23,59,130,07/08/2017,08/08/2017,130,24,51.509506,-0.075459,"Tower Gardens , Tower",0.0,16.7
1353592,424,12038,0,6,405,23,59,755,07/08/2017,08/08/2017,755,24,51.485121,-0.174971,"The Vale, Chelsea",0.0,16.7


After writing this cell I had realised that the only date all three datasets hand in common was the 13th. Now I realise how that changing the format of the date was not necessary I could have just merged the two after the simple modifications were made.

## Analyse